In [1]:
import transformers 
import datasets
import torch
import logging

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset=torch.load("./ms_marco.pt")

In [10]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 101093
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 808731
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 101092
    })
})

In [7]:
test=torch.load('./ms-marco_train_qa.pt')

In [8]:
test

Dataset({
    features: ['id', 'question', 'answer', 'context', 'answer_start', 'answer_end'],
    num_rows: 55578
})

In [4]:
dataset['train'][0]

{'answers': ['The immediate impact of the success of the manhattan project was the only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.'],
 'passages': {'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'passage_text': ['The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.',
   'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.',
   'Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of th

In [103]:
class MyDataset:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data[index]
        return {
            'id': row['id'],
            'question': row['question'],
            'answer': row['answer'],
            'context': row['context'],
            'answer_start': row['answer_start'],
            'answer_end': row['answer_end']
        }

    def get(self, index):
        return self.__getitem__(index)

    def set(self, index, id=None, question=None, answer=None, context=None, answer_start=None, answer_end=None):
        if id is not None:
            self.data[index]['id'] = id
        if question is not None:
            self.data[index]['question'] = question
        if answer is not None:
            self.data[index]['answer'] = answer
        if context is not None:
            self.data[index]['context'] = context
        if answer_start is not None:
            self.data[index]['answer_start'] = answer_start
        if answer_end is not None:
            self.data[index]['answer_end'] = answer_end


In [1]:
def safe_index(arr, value):
    try:
        return arr.index(value)
    except ValueError:
        return -1
def preprocess_dataset(dataset):
    data=[]
    for i in range(len(dataset)):
        # find at what spot is the answer
        index_answer_start = safe_index(dataset[i]['passages']['is_selected'],1)
        if(index_answer_start!=-1):

            # count the characters until the answer start
            # calculate at what point the character starts in the given context
            start=0
            end=0
            for j in range(index_answer_start):
                start=len(dataset[i]['passages']['passage_text'][j])
            end=start+len(dataset[i]['passages']['passage_text'][index_answer_start])
            # set id
            id=dataset[i]['query_id']
            question=dataset[i]['query']
            answer=dataset[i]['answers'][0]
            # join context
            delimiter=' ' 
            context = delimiter.join(dataset[i]['passages']['passage_text'])
            data.append(    {'id': id, 'question': question, 'answer': answer, 'context': context, 'answer_start': start, 'answer_end': end},)
    return MyDataset(data)  

In [106]:
dataset_validation=preprocess_dataset(dataset['validation'])

In [110]:
dataset_validation[0]

{'id': 1102432,
 'question': '. what is a corporation?',
 'answer': 'A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.',
 'context': "A company is incorporated in a specific nation, often within the bounds of a smaller subset of that nation, such as a state or province. The corporation is then governed by the laws of incorporation in that state. A corporation may issue stock, either private or public, or may be classified as a non-stock corporation. If stock is issued, the corporation will usually be governed by its shareholders, either directly or indirectly. Today, there is a growing community of more than 2,100 Certified B Corps from 50 countries and over 130 industries working together toward 1 unifying goal: to redefine success in business. Join the Movement Corporation definition, an association of individuals, created by law or under authority of law, having a continuous existence independent of the existences of it

In [111]:
torch.save(dataset_validation,"ms-marco_valid_qa.pt")

In [118]:
dataset_validation=torch.load('ms-marco_valid_qa.pt')

In [120]:
dataset_validation[0]

{'id': 1102432,
 'question': '. what is a corporation?',
 'answer': 'A corporation is a company or group of people authorized to act as a single entity and recognized as such in law.',
 'context': "A company is incorporated in a specific nation, often within the bounds of a smaller subset of that nation, such as a state or province. The corporation is then governed by the laws of incorporation in that state. A corporation may issue stock, either private or public, or may be classified as a non-stock corporation. If stock is issued, the corporation will usually be governed by its shareholders, either directly or indirectly. Today, there is a growing community of more than 2,100 Certified B Corps from 50 countries and over 130 industries working together toward 1 unifying goal: to redefine success in business. Join the Movement Corporation definition, an association of individuals, created by law or under authority of law, having a continuous existence independent of the existences of it

In [121]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 101093
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 808731
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 101092
    })
})

In [122]:
dataset_train=preprocess_dataset(dataset['train'])

In [139]:
dataset_train[0]['context']

"The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated. The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science. Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade. The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period of the project from 194 … 2-1946 under the control of the U.S. Army Corps of Engineers, under t

In [125]:
torch.save(dataset_train,"ms-marco_train_qa.pt")

In [130]:
dataset_test=preprocess_dataset(dataset['test'])

In [131]:
torch.save(dataset_test,"ms-marco_test_qa.pt")

In [132]:
dataset_test[0]

IndexError: list index out of range